# Code for interacting with API of openFDA

## Plan
In this project, we are looking for device adverse events. We are particularly interested in:
    - Cause of failure
    - Date of failure (ie age of device)
We want to see which is the most common cause for given device to fail and at which stage of use this occurs.
Relevant fields could be (for full reference see https://open.fda.gov/device/event/reference/):
### Event
device_date_of_manufacturer
date_of_event, date_report, date_received
previous_use_code, remedial_action
single_use_flag
### Source
reprocessed_and_reused_flag
### Device
device.generic_name
device.expiration_date_of_device, device.device_age_text
device.implant_flag, device.date_removed_flag
device.manufacturer_d_name, device.manufacturer_d_state, device.manufacturer_d_country
### Patient
patient.sequence_number_outcome, patient.sequence_number_treatment
### Report Text
mdr_text.text, mdr_text.text_type_code
### Reporter Dependent Fields
#### By user facility / importer
report_date
event_location
manufacturer_name, manufacturer_country
manufacturer_g1_name, manufacturer_g1_state
### OpenFDA fields
device_class
### Further interesting fields:
Source: reporter_occupation_code
Device: device.device_operator

In [1]:
import numpy as np
import pandas as pd
import json
import requests

fileName = 'device-event-2016Q1-JSON~\device-event-0001-of-0002.json'
baseurl = 'https://api.fda.gov/device/event.json?'
apikey = ''

In [105]:
# Example of quering, for complete guide go to: https://open.fda.gov/api/
query = 'search=(device.generic_name:"stent"+AND+reprocessed_and_reused_flag:"Y"+AND+date_of_event:' + \
'["20150324"+TO+"20170324"]+AND+(_exists_:device_date_of_manufacturer+AND+_exists_:date_of_event))&limit=10'
# query2 = '_exists_:search=()'
q1 = baseurl + 'api_key=' + apikey + '&' + query

In [106]:
dq1 = requests.get(q1)
# dq1.json()['results']
data = json.loads(dq1.text)
results = data['results']

In [5]:
# Can also spare some structuring effort when loading data by calling normalization method
# dftest = pd.io.json.json_normalize(results)

In [107]:
results[1]

{'adverse_event_flag': 'N',
 'date_facility_aware': '20161103',
 'date_manufacturer_received': '20161210',
 'date_of_event': '20161103',
 'date_received': '20161128',
 'date_report': '20170103',
 'device': [{'brand_name': 'SMART CONTROL, ILIAC 9X40',
   'catalog_number': 'C09040SL',
   'date_received': '20161128',
   'date_removed_flag': '',
   'date_returned_to_manufacturer': '20161213',
   'device_age_text': 'DA',
   'device_availability': 'Device was returned to manufacturer',
   'device_evaluated_by_manufacturer': 'R',
   'device_event_key': '',
   'device_operator': 'HEALTH PROFESSIONAL',
   'device_report_product_code': 'NIO',
   'device_sequence_number': ' 1.0',
   'generic_name': 'STENT, ILIAC',
   'implant_flag': '',
   'lot_number': '17556383',
   'manufacturer_d_address_1': 'CIRCUITO INT NTE 1820',
   'manufacturer_d_address_2': '',
   'manufacturer_d_city': 'JUAREZ',
   'manufacturer_d_country': 'MX',
   'manufacturer_d_name': 'CORDIS DE MEXICO',
   'manufacturer_d_postal_c

In [108]:
# Fields of Interest
fois_result = ['device_date_of_manufacturer', 'date_of_event']
fois_device = ['generic_name', 'expiration_date_of_device', 'device_age_text', 'implant_flag', 'date_removed_flag', \
               'manufacturer_d_name', 'manufacturer_d_state', 'manufacturer_d_country']
fois_patient = ['sequence_number_outcome', 'sequence_number_treatment']
fois_mdrText = ['text', 'text_type_code']
fois_openfda = ['device_name', 'device_class', 'medical_specialty_description']

# device = data['results'][0]['device'][0]
device = [x['device'][0] for x in data['results']]
# patient = data['results'][0]['patient'][0]
patient = [x['patient'][0] for x in data['results']]
# mdrText = data['results'][0]['mdr_text'][0] # there may be more items in the list! 
mdrText = [x['mdr_text'] for x in data['results']]
#mdrText = [y['text'] for y in [x['mdr_text'][0] for x in data['results']]]
# openfda = data['results'][0]['device'][0]['openfda']
openfda = [x['device'][0]['openfda'] for x in data['results']]

In [109]:
fillDic = {'mdr_text_key': '', 'patient_sequence_number': '', 'text': np.nan, 'text_type_code': np.nan}
a = [x[0] if len(x) > 0 else fillDic for x in mdrText]
b = [x[1] if len(x) > 1 else fillDic for x in mdrText]

In [112]:
df_results = pd.DataFrame(results, index = range(len(results)), columns = fois_result)
df_openfda = pd.DataFrame(openfda, index = range(len(results)),columns = fois_openfda)
df_device = pd.DataFrame(device, index = range(len(results)),columns = fois_device)
df_patient = pd.DataFrame(patient, index = range(len(results)),columns = fois_patient)
# df_mdrText = pd.DataFrame(mdrText, index = range(len(results)),columns = fois_mdrText)

# df = pd.concat([df_device, df_patient, df_mdrText, df_openfda], axis = 1)

In [116]:
a = pd.DataFrame(a, index = range(len(results)),columns = fois_mdrText)
b = pd.DataFrame(b, index = range(len(results)),columns = fois_mdrText)
df_mdrText = pd.concat([a, b], axis = 1)
df = pd.concat([df_results, df_device, df_patient, df_mdrText, df_openfda], axis = 1)

In [117]:
df

,device_date_of_manufacturer,date_of_event,generic_name,expiration_date_of_device,device_age_text,implant_flag,date_removed_flag,manufacturer_d_name,manufacturer_d_state,manufacturer_d_country,sequence_number_outcome,sequence_number_treatment,text,text_type_code,text,text_type_code,device_name,device_class,medical_specialty_description
0,20140110,20150620,ENDOVASCULAR STENT GRAFT,20160110,DA,,,LOMBARD MEDICAL LTD.,,UK,[8.],[],(B)(4).,Description of Event or Problem,DHR REVIEW: A FULL REVIEW OF THE DEVICE HISTOR...,Additional Manufacturer Narrative,"System, Endovascular Graft, Aortic Aneurysm Tr...",3,Unknown
1,20160901,20161103,"STENT, ILIAC",NaN,DA,,,CORDIS DE MEXICO,,MX,[8.],[],REVIEW OF LOT 17556383 REVEALED NO ANOMALIES D...,Additional Manufacturer Narrative,ADDITIONAL INFO RECEIVED INDICATED THAT THE PR...,Additional Manufacturer Narrative,"Stent, Iliac",3,Unknown
2,20150821,20161219,"STENT, CORONARY, DRUG-ELUTING",NaN,,,,MEDTRONIC IRELAND,,,[8.],[],IT WAS REPORTED THAT A PHYSICIAN USED AN ENDEA...,Description of Event or Problem,.,Additional Manufacturer Narrative,Coronary Drug-Eluting Stent,3,Unknown
